# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [2]:
# Run this cell first!

from helpers import Map, load_map_10, load_map_40, show_map
import math

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Map Basics
The map below shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: **`intersections`** and **`roads`**

In [ ]:
map_10 = load_map_10()
show_map(map_10)

In [3]:
## Let's do the bigger one..

map_40 = load_map_40()
show_map(map_40)

---
# 1. Intersections(Nodes)

The `intersections` are represented as a dictionary. 

In this example, there are 40 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [ ]:
map_40.intersections # nodesssss

In [ ]:
map_40.intersections[5]

In [ ]:
map_40.intersections[34]

# 2. Roads(expanding paths)

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [ ]:
len(map_40.roads)

In [ ]:
map_40.roads[5]

In [ ]:
map_40.roads[34]

# 3. Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map()` function which generated this map also takes a few optional parameters which might be useful for visualizaing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [4]:
# run this code, note the effect of including the optional parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5, 16,37,12, 34])

---
# The Algorithm
### Writing your algorithm
The algorithm written will be responsible for **generating a `path`** like the one passed into `show_map` above. In fact, when called with the same **map, start and goal**, as above your algorithm should produce the path `[5, 16, 37, 12, 34]`. However you must complete several methods before it will work.

```bash
> PathPlanner(map_40, 5, 34).path
[5, 16, 37, 12, 34]
```

In [ ]:
'''# A* search Implementation.

from math import sqrt

def shortest_path(M, start, goal):
    """Returns the lowest cost path from start to goal."""
    
    pos = M.intersections
    M = M.roads
    
    closed = set()
    opened = set([start])
    came_from = {}
    num_nodes = len(M)
    g_scores = [99999 for _ in range(num_nodes)]
    f_scores = [99999 for _ in range(num_nodes)]
    g_scores[start] = 0.0
    f_scores[start] = heuristic(start, goal, pos)
    
    while opened:
        current = get_next_node(opened, f_scores)
        if current == goal:
            print("f_score:", f_scores[current])
            print("g_score:", g_scores[current])
            return reconstruct_path(came_from, current, start)
        opened.remove(current)
        closed.add(current)
        neighbors = M[current]
        for neighbor in neighbors:
            if neighbor in closed:
                continue
            if neighbor not in opened:
                opened.add(neighbor)
            
            tentative_g = g_scores[current] + distance(current, neighbor, pos)
            if tentative_g >= g_scores[neighbor]:
                continue
            came_from[neighbor] = current
            g_scores[neighbor] = tentative_g
            f_scores[neighbor] = g_scores[neighbor] + heuristic(neighbor, goal, pos)
    return None
    
    

def get_next_node(opened, f_scores):
    best_idx = None
    lowest_f = 9999999                         ######float('inf') ??
    for node in opened:
        f = f_scores[node]
        if f < lowest_f:
            best_idx = node
            lowest_f = f
    return best_idx

def reconstruct_path(came_from, current, start):
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path





def distance(start, goal, pos):
    x1, y1 = pos[start]
    x2, y2 = pos[goal]
    return sqrt( (x2-x1)**2 + (y2-y1)**2 )

def heuristic(start,goal,pos):
    return distance(start, goal, pos)


'''


In [5]:
# Do not change this cell
# When you write your methods correctly, this cell will execute without problems
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        self.map = M
        self.start= start
        self.goal = goal
        
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None #####

        
    def run_search(self):
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running: PathPlanner.set_map(start_node)") ###########
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running: PathPlanner.set_start(start_node)") ##
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running: PathPlanner.set_goal(start_node)") ####
        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet() 
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()
        
        
        while not self.is_open_empty(): # run 'if the openSet is NOT empty'. 
            current = self.get_current_node() #Returns the node in the openSet with the "lowest value of f(node)"

            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))] #
                return self.path
            else:
                self.openSet.remove(current) # goal?
                self.closedSet.add(current) # frontier?
                
            for neighbor in self.get_neighbors(current): ###
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.
                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                # The distance from start to a neighbor; the "dist_between" function may vary as per the solution requirements.
                if self.get_tenative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.
                self.record_best_path_to(current, neighbor) # This path is the best until now. Record it!
                
                
        print("No Path Found")
        self.path = None
        return(False)        
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current] ##a list?
        while current in self.cameFrom.keys():
            current = self.cameFrom[current] ##it's gonna be a list of possible connectivity? 
            total_path.append(current)
        return total_path    
        
    def get_path(self):
        """ Reconstructs path after search """
        if self.path:
            return self.path 
        else :
            self.run_search() ##########
            return self.path
    
    
################################################################################################################################
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        
        self.path = self.run_search() if self.map and self.start and self.goal else None

Create the following methods:

In [6]:
# no.1

def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """
    # TODO: Return True if the "open set" is empty. False otherwise.
    if len(self.openSet)==0:
        return True
    else:
        return False


## [note]
max() or min() function using 'key' and lambda expression?

#### Lambda
- It is used for creating small, one-time and anonymous function objects: 
  - `lambda (arguments) : (expression)`
  - lambda operator can have any number of arguments, but it can have only one expression. It cannot contain any statements and it returns a **function object** which can be assigned to any variable.
- Let's say function name is **add()**, it expects 2 arguments x and y and returns their sum.
- In this case the **function object** is assigned to the **add** variable.

```
def add(x, y): 
    return x + y
  
add(2, 3)  # Output: 5
----------------------------
add = lambda x, y : x + y 
print(add(2, 3)) # Output: 5
```
> Mostly lambda functions are passed as parameters to a function which expects a **function objects** as parameter like..
- map(), reduce(), filter()
  

**`map(function_object, iterable1, iterable2,...)`**
- It executes the **function_object** for each element in the sequence and returns a **list** of the elements modified by the function object. In Python3, map function returns an iterator or map object. We can force convert the map-output such as `list(map object)` 

```
ex1>###################################################
def multiply2(x):
  return x*2
map(multiply2, [1, 2, 3, 4])  # Output [2, 4, 6, 8]
-------------------------------------------------------
map(lambda x : x*2, [1, 2, 3, 4]) #Output [2, 4, 6, 8]

ex2>#################
list_a = [1, 2, 3]
list_b = [10, 20, 30]
---------------------------------------------------------------  
map(lambda x, y: x + y, list_a, list_b) # Output: [11, 22, 33]

ex3>######################################################################
two_dict = [{'name': 'python', 'points': 10}, {'name': 'java', 'points': 8}]
-------------------------------------------------  
map(lambda x : x['name'], two_dict) 
map(lambda x : x['points']*10,  two_dict) 
map(lambda x : x['name'] == "python", two_dict) # Output: ['python', 'java'], Output: [100, 80], Output: [True, False]
```

**`filter(function_object, iterable)`**
- filter function expects two arguments, function_object and an iterable(only one). **function_object returns a boolean value**. function_object is called for each element of the iterable and filter returns only those element for which the function_object returns true. filter function in Python3 returns a filter object or the iterator which gets lazily evaluated, so Converts the filter obj to a list..

```
a = [1, 2, 3, 4, 5, 6]
filter(lambda x : x % 2 == 0, a)      # Output: [2, 4, 6]

two_dict = [{'name': 'python', 'points': 10}, {'name': 'java', 'points': 8}]
filter(lambda x : x['name'] == 'python', two_dict)      # Output: [{'name': 'python', 'points': 10}]

list_a = [1, 2, 3, 4, 5]
filter_obj = filter(lambda x: x % 2 == 0, list_a) 
even_num = list(filter_obj) 
print(even_num)      # Output: [2, 4]
```

#### min(), max() function using 'key'
- like a filtering ???
- To modify the object before comparing based on a particular attribute/index you've to use the **key** argument.
- For example, suppose you've a list of numbers in **string** form, but you want to compare those items by their **integer** value.

```
lis = ['1','100','111','2']
max(lis, key=lambda x: int(x))      # Output: '111'
```
- By default max will will compare the items by the first index, if the first index is same then it'd compare the second index. But, what if you wanted to compare each item by the value at index 1?

```
lis = [(1,'a'),(3,'c'), (4,'e'), (-1,'z')]
max(lis, key = lambda x: x[1])     # Output: (-1,'z')
```
- Strongly simplified version of `max()` 

What is max()??
```
def max(items, key=lambda x: x):
    current = item[0]
    for i in items:
        if key(i) > key(current):
            current = i
    return current
```


In [ ]:
k={'g':1,'h':6,'i':8}; k.keys()

In [7]:
# no.2
# Basically 'current' is the node with the minimum f-score in this iteration.
# the search will stop when current is the goal node.
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""
    # TODO: Return the node in the "open set" with the lowest value of f(node).
    #low_f = sorted(self.fScore.keys())
    #low_f_value = self.fScore[low_f[0]]
    #if low_f_value in self.openSet:
    #    return low_f_value
    box ={} #hold the result of fScore function for each node in openSet
    for i in self.openSet:
        box[i] = self.calculate_fscore(i)
    current_node = min(box, key=box.get)
    return current_node
    

In [8]:
#no.3

def get_neighbors(self, node):
    """Returns the neighbors of a node"""
    # TODO: Return the neighbors of a node
    return set(self.map.roads[node])


## [note]
### Get() method for dictionaries
- In python dictionaries, following is a conventional method to access a value for a key

```
dic = {"A":1, "B":2}
print(dic["A"])
print(dic["C"])
```
- The problem that arises here is that the 3rd line of the code returns a key error.
- `get(key, default = None)` method is used to avoid such situations. This method returns the value for the given key, if present in the dictionary. If not, then **it will return None** (if get() is used with only one argument).

```
dic = {"A":1, "B":2}
print(dic.get("A"))
print(dic.get("C"))
print(dic.get("C","Not Found ! "))  #Output: 1, #Output: None, #Output: Not Found !
```

In [9]:
#no.4

def get_tenative_gScore(self, current, neighbor):
    """Returns the tenative g Score of a node"""
    # TODO: Return the g Score of the current node plus distance from the current node to it's neighbors
    return(self.get_gScore(current) + self.distance(current, neighbor))


In [10]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
#no.5

def get_gScore(self, node):
    """Returns the g Score of a node"""
    # TODO: Return the g Score of a node
    gg = self.gScore.get(node)
    if gg == None:
        return float('inf')
    else:
        return gg


In [11]:
#no.6

def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    self.cameFrom[neighbor] = current 
    # current is the node with the minimum f-score in this iteration. If the implementation is correct and the heuristic 
    # used is consistent, this means that at this point it is guaranteed that we already found the shortest path from 
    # the start to current. This is why current is added to the closed set and ignored if it shows up again during 
    # the execution. This is also the reason why the search stops when current is the goal node.
    # So, after finding current, the algorithm evaluates all its neighbors that are not already in the closed set. 
    # The record_best_path function is called when the neighbor being evaluated appears for the first time during the execution,
    # or if, at this iteration, it is reached from a path that is shorter than the best one found before.
    
    self.gScore[neighbor] = self.get_tenative_gScore(current, neighbor) 
    self.fScore[current] = self.calculate_fscore(current)


In [12]:
#no.0-1

def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    return(set())

In [13]:
#no.0-2

def create_openSet(self):
    """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
    that are not evaluated yet. Initially, only the start node is known."""
    if self.start != None:
        # TODO: return a data structure suitable to hold the set of currently discovered nodes 
        # that are not evaluated yet. Make sure to include the start node.
        init_node = set()
        init_node.add(self.start)
        return(init_node)
    
    raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)")

In [14]:
#no.0-3

def create_cameFrom(self):
    """Creates and returns a data structure that shows which node can most efficiently be reached from another,
    for each node."""
    # TODO: return a data structure that shows which node can most efficiently be reached from another, for each node.
    return({})    

## [note]
what is the point of having a variable store an infinite value in a program? 

It acts as an unbounded upper value for comparison. This is useful for finding lowest values for something. For example, calculating path route costs when traversing trees(Finding the "cheapest" path in a list of options:)
```
lowest_path_cost = float('inf')
path_costs = [1, 100, 2000000000000, 50]
for path in path_costs:
    if path < lowest_path_cost:
        lowest_path_cost = path
lowest_path_cost
```

if you didn't have `float('Inf')` available to you, what value would you use for **the initial lowest_path_cost**? Would 9999999 be enough -- `float('Inf')` removes this guesswork.

In [15]:
#no.0-4

def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # TODO:  a data structure that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value should be set to infinity.
    fscore = {}
    #f_score[self.calculate_fscore(self.start) + 1.0] = self.start
    #return f_score  

    for i in range(len(self.map.intersections)):
        fscore[i] = float('inf')
    fscore[self.start] = self.heuristic_cost_estimate(self.start)
    return fscore


In [16]:
#no.0-5

def create_gScore(self):
    """Creates and returns a data structure that holds the cost of getting from the start node to that node, for each node.
    The cost of going from start to start is zero."""
    # TODO: a data structure that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values should be set to infinity.
    gscore = {}
    for i in range(len(self.map.intersections)):
        gscore[i] = float('inf')
    gscore[self.start] = 0
    return gscore


In [ ]:
###############################################################################################################################

In [17]:
#no.0-6 # for run_search() value error...saying...[if self.map == None]
#"Must create map before running search. Try running:::::::::::::::::::: PathPlanner.set_map(start_node)"

def set_map(self, M):
    """Method used to set map attribute """
    self._reset(self)
    self.start = None
    self.goal = None
    # TODO: Set map to new value.
    self.map = M


In [18]:
#no.0-6 # for run_search() value error...saying...[if self.start == None]
#"Must create map before running search. Try running:::::::::::::::::::::: PathPlanner.set_start(start_node)"

def set_start(self, start):
    """Method used to set start attribute """
    self._reset(self)
    # TODO: Set start value. Remember to remove goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    self.goal = None
    self.closedSet = None
    self.openSet = None
    self.cameFrom = None
    self.gScore = None
    self.fScore = None
    self.path = None
    self.start = start

In [19]:
#no.0-6 # for run_search() value error...saying...[if self.goal == None]
#"Must create map before running search. Try running::::::::::::::::::::: PathPlanner.set_goal(start_node)"

def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # TODO: Set goal value.
    self.goal = goal


In [ ]:
###############################################################################################################################

In [20]:
# calculation-01

def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""
    # TODO: Compute and return the Euclidean L2 Distance
    (x1, y1) = self.map.intersections[node_1]
    (x2, y2) = self.map.intersections[node_2]
    dist = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return dist


In [21]:
# calculation-02

def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node """
    # TODO: Return the heuristic cost estimate of a node
    (x1, y1) = self.map.intersections[node]
    (x2, y2) = self.map.intersections[self.goal]
    dist = math.sqrt((x2-x2)**2 + (y2-y1)**2)
    return dist


In [22]:
# calculation-03

def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    # TODO: Calculate and returns the f score of a node. 
    # REMEMBER F = G + H
    f = self.get_gScore(node) + self.heuristic_cost_estimate(node)
    return f


In [23]:
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
#PathPlanner._reset = _reset ??????????????????????????????????????????? in the class definition
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.get_tenative_gScore = get_tenative_gScore
PathPlanner.is_open_empty = is_open_empty
PathPlanner.distance = distance
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

In [24]:
planner = PathPlanner(map_40, 5, 34)
path = planner.path
if path == [5, 16, 37, 12, 34]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

great! Your code works for these inputs!


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, submit by pressing the Submit button in the lower right!

In [25]:
from test import test

test(PathPlanner)

All tests pass! Congratulations!


## Questions

**Instructions**  Answer the following questions in your own words. We do not you expect you to know all of this knowledge on the top of your head. We expect you to do research and ask question. However do not merely copy and paste the answer from a google or stackoverflow. Read the information and understand it first. Then use your own words to explain the answer.

- How would you explain A-Star to a family member(layman)?

** ANSWER **: When taking a route, we can have multiple options and each might have upsides and downsides. Some routes might have shorter distance to travel but cost more time and money due to heavy traffic and other routes might be longer path but faster. We want to choose the best route and this is where A-Star search comes in. A-Star does not actually know or care about how these costs or distances are decided, but A-Star already receives the cost (or distance) between every pair of neighbor nodes, and tries to find the minimum cost path from the start to the goal.

> In this particular problem that involves a road map, it can be the fuel cost, the physical distance, the time that takes to go from one node to its neighbor, a combination of all or some of these costs, or any other thing. The **heuristic cost estimation** is used to orient the search: i.e. to avoid exploring as many nodes as possible, **reducing computational costs**.

- How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

** ANSWER **: Both **Best First Search**, and **Uniform cost Search** tries to get the cheapest path, but the difference is in heuristic function. The **best First Search** is informed search(with heuristic function: estimating how close to goal each of the next states will be from the current state) while **Uniform cost Search** is uninformed search so does not use the heuristic function. However, **Best First Search** differ from **A-Star Search** as **A-Star** visits next state based on heristics **f(n) = h + g** where h component is same heuristics applied in Best first search but **g component is path from the initial state to the particular state**; therefore, it doesn't chooses next state only with lowest heuristics value but one that gives lowest value when considering it's heuristics and cost of getting to that state.

> The main differences among all three algorithms are about **what kind of information is used** during the search, and the guarantees about finding the optimal solution:
- https://stackoverflow.com/questions/44151713/what-is-the-difference-between-uniform-cost-search-and-best-first-search-methods
- https://stackoverflow.com/questions/34244452/whats-the-difference-between-best-first-search-and-a-search


- What is a heuristic?

** ANSWER **: It's the distance estimation to our destination 

- What is a consistent heuristic?

** ANSWER **: **h_distance(current, goal)** <= g_cost(current, neighbor) + h_distance(neighbor, goal)
- The estimated distance (from current state to the destination) is less than or equal to the cost (from current state to a neighbor node) plus the estimated distance (from its neighbor node to the destination).

- What is a admissible heuristic? 

** ANSWER **: h(current, goal) <= Cost-to-goal 
- Some heuristic functions are admissible when it never overestimates the cost of reaching the destination. Its estimated cost is never more than the actual cost(from current state to the goal). If it overestimate the cost, the algorithm  would not explore the path and try to find another path to the goal.

- ___ admissible heuristic are consistent.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **: Some

- ___ Consistent heuristic are admissible.
*CHOOSE ONE*
    - All
    - Some
    - None
    
** ANSWER **: All